# Firebolt Core Examples

This notebook demonstrates how to use the Firebolt Python SDK with Firebolt Core, a Docker-based version of Firebolt that can be run locally or remotely without authentication requirements.

Firebolt Core is ideal for:
- Local development and testing
- CI/CD environments
- Demonstration and educational purposes
- Environments where authentication is handled elsewhere

The examples in this notebook cover both synchronous and asynchronous connectivity, and demonstrate key SDK features including:
- Connecting to Firebolt Core instances
- Executing SQL queries
- Using parameterized queries
- Handling multi-statement queries
- Processing streaming results
- Error handling
- Unit testing

## Prerequisites & Setup

### Running Firebolt Core

Firebolt Core can be run locally using Docker with the following command:

```bash
docker run -p 3473:3473 firebolt/firebolt-core:latest
```

You can also customize the port and other settings:

```bash
docker run -p 8080:3473 -e FIREBOLT_SERVER_PORT=3473 firebolt/firebolt-core:latest
```

For this notebook, we'll assume Firebolt Core is running locally on the default port (3473).

## Connecting to Firebolt Core

There are two ways to connect to Firebolt Core:

1. **Synchronous connection** - using the standard DB-API 2.0 interface
2. **Asynchronous connection** - using the async equivalent API

Both methods use the `FireboltCore` authentication class, which doesn't require actual credentials.

### Synchronous Connection

In [1]:
from firebolt.db import connect
from firebolt.client.auth import FireboltCore

# Create a FireboltCore auth object - no credentials needed
# You can specify a custom URL or use the default (http://localhost:3473)
auth = FireboltCore()  # Default: http://localhost:3473
# Or with custom URL: auth = FireboltCore(url="http://localhost:8080")

# Connect to Firebolt Core
# The database parameter defaults to 'firebolt' if not specified
connection = connect(auth=auth, database="firebolt")

# Create a cursor
cursor = connection.cursor()

# Execute a simple test query
cursor.execute("SELECT 1")

# Fetch and display results
result = cursor.fetchall()
print(f"Query result: {result}")

# Get column names
print(f"Column names: {cursor.description[0][0]}")

# Show connection parameters (filtered for Firebolt Core)
print(f"Connection URL: {connection.engine_url}")

# Close connection when done
connection.close()

Query result: [[1]]
Column names: ?column?
Connection URL: http://localhost:3473


### Asynchronous Connection

Firebolt SDK also supports asynchronous connections, which is useful for applications that need non-blocking database operations:

In [ ]:
from firebolt.async_db import connect as async_connect
from firebolt.client.auth import FireboltCore


async def run_async_query():
    # Create a FireboltCore auth object
    auth = FireboltCore()

    # Connect to Firebolt Core asynchronously
    connection = await async_connect(auth=auth, database="firebolt")

    # Create a cursor
    cursor = connection.cursor()

    # Execute a simple test query
    await cursor.execute("SELECT 2")

    # Fetch and display results
    result = await cursor.fetchall()
    print(f"Async query result: {result}")

    # Get column names
    print(f"Column names: {cursor.description[0][0]}")

    # Close connection when done
    await connection.aclose()

    return result


# Run the async function
await run_async_query()

## Working with Database Objects

Let's explore how to create and manipulate database objects in Firebolt Core. We'll create tables, insert data, and query the results.

In [ ]:
# Reuse the connection from earlier or create a new one
from firebolt.db import connect
from firebolt.client.auth import FireboltCore

# Connect to Firebolt Core
auth = FireboltCore()
connection = connect(auth=auth, database="firebolt")
cursor = connection.cursor()

# Create a new table
create_table_sql = """
CREATE TABLE IF NOT EXISTS example_table (
    id INT,
    name TEXT,
    value FLOAT,
    created_at TIMESTAMP
)
"""

cursor.execute(create_table_sql)
print("Table created successfully!")

In [ ]:
# Insert data into the table
insert_data_sql = """
INSERT INTO example_table (id, name, value, created_at)
VALUES 
    (1, 'Item 1', 10.5, '2023-01-01 12:00:00'),
    (2, 'Item 2', 20.75, '2023-01-02 14:30:00'),
    (3, 'Item 3', 15.25, '2023-01-03 09:45:00')
"""

cursor.execute(insert_data_sql)
print("Data inserted successfully!")

In [ ]:
# Query the data
query_sql = """
SELECT * FROM example_table
ORDER BY id
"""

cursor.execute(query_sql)
results = cursor.fetchall()

# Display results
print("Query Results:")
for row in results:
    print(row)

# Get column information
columns = [desc[0] for desc in cursor.description]
print("\nColumns:", columns)

## Parameterized Queries

Parameterized queries are a crucial feature for building secure applications. They help prevent SQL injection attacks and make code more readable and maintainable. Firebolt Core supports parameterized queries using the standard DB-API 2.0 parameter style.

In [ ]:
# Example of a parameterized query
parameterized_query = """
SELECT * FROM example_table 
WHERE id > ? AND value < ?
ORDER BY value DESC
"""

# Parameter values
params = (1, 20.0)

# Execute with parameters
cursor.execute(parameterized_query, params)

# Fetch and display results
param_results = cursor.fetchall()
print("Parameterized Query Results:")
for row in param_results:
    print(row)

## Multi-Statement Queries

Firebolt Core supports executing multiple SQL statements in a single query. This is particularly useful for running complex scripts, setting up multiple objects at once, or performing related operations together.

In [ ]:
# Multi-statement query example
multi_statement_query = """
-- First statement creates a temporary table
CREATE TABLE IF NOT EXISTS temp_stats AS 
SELECT 
    AVG(value) as avg_value,
    MAX(value) as max_value,
    MIN(value) as min_value
FROM example_table;

-- Second statement queries the temp table
SELECT * FROM temp_stats;

-- Third statement cleans up
DROP TABLE temp_stats;
"""

# Execute multiple statements
cursor.execute(multi_statement_query)

# The result will be from the last SELECT statement
multi_results = cursor.fetchall()
print("Multi-Statement Query Results:")
for row in multi_results:
    print(row)

# Column names
print("\nColumns:", [desc[0] for desc in cursor.description])

## Streaming Results

For large result sets, it's often more efficient to stream results rather than fetching them all at once. Firebolt Core supports streaming results using the iterator interface of the cursor.

In [ ]:
# First, let's create a larger dataset for demonstration
cursor.execute("DROP TABLE IF EXISTS streaming_demo")
cursor.execute(
    """
CREATE TABLE streaming_demo AS
SELECT 
    number as id,
    CONCAT('Item ', CAST(number AS VARCHAR)) as name,
    number * 1.5 as value,
    TIMESTAMP '2023-01-01 12:00:00' + INTERVAL number HOUR as created_at
FROM numbers(1000)
"""
)

print("Created larger demo table with 1000 rows")

# Now let's stream the results
cursor.execute("SELECT * FROM streaming_demo ORDER BY id")

# Stream results instead of fetching all at once
print("Streaming first 5 results:")
for i, row in enumerate(cursor):
    print(row)
    if i >= 4:  # Show just first 5 rows
        print("...")
        break

print(
    f"\nTotal rows available: {cursor.rowcount if cursor.rowcount >= 0 else 'unknown'}"
)

# Using fetchmany
cursor.execute("SELECT * FROM streaming_demo WHERE id % 100 = 0 ORDER BY id")
print("\nFetching in batches of 2:")
batch = cursor.fetchmany(2)
while batch:
    print(f"Batch: {batch}")
    batch = cursor.fetchmany(2)

In [ ]:
# Cleanup - drop demo tables and close connection
try:
    cursor.execute("DROP TABLE IF EXISTS example_table")
    cursor.execute("DROP TABLE IF EXISTS streaming_demo")
    print("Cleaned up demo tables")
finally:
    # Close the connection to release resources
    if "connection" in locals() and connection:
        connection.close()
        print("Connection closed")

print("Notebook execution complete!")